# Imports

In [102]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)

movies = pd.read_csv('https://raw.githubusercontent.com/nchichilidze/RS-with-GE/main/preprocessed_movielens_1m/movies.csv')
tags = pd.read_csv('https://raw.githubusercontent.com/nchichilidze/RS-with-GE/main/preprocessed_movielens_1m/tags.csv')
ratings = pd.read_csv('https://raw.githubusercontent.com/nchichilidze/RS-with-GE/main/preprocessed_movielens_1m/ratings.csv')

movies = movies.iloc[: , 1:]
tags = tags.iloc[: , 1:]

imdb_ratings = pd.read_csv("title.ratings.tsv", sep='\t')
imdb_titles = pd.read_csv("title.basics.tsv", sep='\t')

/Users/nutsa/.pyenv/versions/3.7.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [103]:
# drop non movie titleTypes 

non_movie_titleTypes = ['tvEpisode', 'tvSeries', 'tvMiniSeries', 'video', 'videoGame', 'tvPilot']

for titleType in non_movie_titleTypes: 
    imdb_titles = imdb_titles[imdb_titles['titleType'] != titleType]

In [104]:
movies_full = movies.copy(deep=True)
movies_full['title'] = movies_full['title'].apply(lambda x: x.lower())

imdb_full = imdb_ratings.copy(deep=True)
imdb_full = pd.merge(imdb_full, imdb_titles, on='tconst')

imdb_full = imdb_full.rename(columns={'primaryTitle': 'title'})
imdb_full = imdb_full.rename(columns={'startYear': 'year'})
imdb_full = imdb_full.rename(columns={'genres': 'imdb_genres'})
imdb_full = imdb_full.drop(columns='endYear')
imdb_full['title'] = imdb_full['title'].apply(lambda x: x.lower())

In [105]:
movies_full = movies_full.astype({'title':'string'})

imdb_full = imdb_full.replace('\\N', None)
imdb_full = imdb_full.astype({'title':'string', 
                              'year':'int64'})


# Match IMDB & MovieLens

In [106]:
ml_unmatched = movies_full.copy(deep='True')
ml_unmatched = ml_unmatched.drop(columns = ['genres'])

imdb = imdb_full.copy(deep='True')
imdb = imdb.drop(columns = ['averageRating','numVotes','titleType','isAdult','runtimeMinutes', 'imdb_genres'])

unmatched_movie_ids = ml_unmatched['movie_id']

print("movies to match: " + str(ml_unmatched.shape))

movies to match: (3883, 3)


In [107]:
import string 
from string import punctuation

def reposition_movielens_article(title): 
    articles = {', the', ', a', ', an', ', le', ', la', ', l\'', ', el', ', dir', ', der'}
    for article in articles: 
        if(title[-len(article):] == article):
            title = article[2:] + ' ' + title[:title.index(article)]
    return title 

def remove_movielens_articles(title):
    new_title = title
    articles = {', the', ', a', ', an', ', le', ', la', ', l\'', ', el', ', dir', ', der'}
    for article in articles: 
        if article in title: 
            new_title = title[:title.index(article)]
    return new_title 

def remove_imdb_articles(title): 
    new_title = title
    articles = {'the ', 'a ', 'an ', 'le ', 'la ', 'l\' ', 'el ', 'dir', 'der'}
    for article in articles: 
        if article in title: 
            new_title = title[title.index(article) + len(article):] 
    return new_title

def remove_punctuation(title): 
    new_title = title
    if (any(p in title for p in punctuation)):
        new_title = title.translate(str.maketrans('', '', string.punctuation))
    return new_title 

def remove_numbers(title):
    no_digits = []
    for i in title:
        if not i.isdigit():
            no_digits.append(i)
    return ''.join(no_digits)

def remove_secondary(title):
    new_title = title
    if ('(' in title and ')' in title):
        new_title = title[: title.index('(')-1]
    
    return new_title 

def extract_secondary(title):
    new_title = title
    if ('(' in title and ')' in title):
        new_title = title[title.index('(') + 1: title.index(')')]
    return new_title  

In [108]:
# match on title + year 
match = pd.merge(ml_unmatched, imdb, on=['title', 'year'])
matched_df_full = match.copy(deep='True')


matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* title + year")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# match on original title + year 
imdb['originalTitle'] = imdb['originalTitle'].apply(lambda x: x.lower())

match = pd.merge(ml_unmatched, imdb, 
                 left_on=['title', 'year'],
                 right_on=['originalTitle', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* original title + year")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

* title + year
new matches: (2455, 5)
all matches: (2455, 5)
left to match: (1460, 3)
* original title + year
new matches: (36, 6)
all matches: (2491, 7)
left to match: (1424, 3)


In [109]:
# match on original title + (year + 1)
ml_unmatched['year+1'] = ml_unmatched['year'] + 1
ml_unmatched['year-1'] = ml_unmatched['year'] - 1


match = pd.merge(ml_unmatched, imdb, 
                 left_on=['title', 'year+1'],
                 right_on=['originalTitle', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* original title + (year+1) ")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# match on title + (year+1)

ml_unmatched['year+1'] = ml_unmatched['year'] + 1
ml_unmatched['year-1'] = ml_unmatched['year'] - 1


match = pd.merge(ml_unmatched, imdb, 
                 left_on=['title', 'year+1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* title + (year+1) ")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

* original title + (year+1) 
new matches: (65, 9)
all matches: (2556, 11)
left to match: (1361, 5)
* title + (year+1) 
new matches: (1, 8)
all matches: (2557, 11)
left to match: (1360, 5)


In [110]:
# match on a year that is 1 off 
# match on original title 
match = pd.merge(ml_unmatched, imdb, 
                 left_on=['title', 'year-1'],
                 right_on=['originalTitle', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("original title + year - 1")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# match on a year that is 1 off 
# match on title 
match = pd.merge(ml_unmatched, imdb, 
                 left_on=['title', 'year-1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("title + year - 1")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

original title + year - 1
new matches: (82, 9)
all matches: (2639, 11)
left to match: (1278, 5)
title + year - 1
new matches: (8, 8)
all matches: (2647, 11)
left to match: (1270, 5)


In [111]:
# fix the article notation movielens has
# The notation is i.e. Contender, The 

ml_unmatched_articles = ml_unmatched.copy(deep='True')

ml_unmatched_articles['title'] = ml_unmatched_articles['title'].apply(lambda x: reposition_movielens_article(x))

match = pd.merge(ml_unmatched_articles, imdb, 
                 left_on=['title', 'year'],
                 right_on=['originalTitle', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("fixed article + original title")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

ml_unmatched_articles = ml_unmatched_articles[ml_unmatched_articles['movie_id'].isin(unmatched_movie_ids)]

# article notation change but match with title instead of originalTitle 

match = pd.merge(ml_unmatched_articles, imdb, 
                 left_on=['title', 'year'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)


matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("fixed article + title")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

fixed article + original title
new matches: (715, 8)
all matches: (3362, 11)
left to match: (560, 5)
fixed article + title
new matches: (19, 7)
all matches: (3381, 11)
left to match: (541, 5)


In [112]:
# let's try the +- year again 

ml_unmatched_articles = ml_unmatched_articles[ml_unmatched_articles['movie_id'].isin(unmatched_movie_ids)]

ml_unmatched_articles['year+1'] = ml_unmatched_articles['year']+1
ml_unmatched_articles['year-1'] = ml_unmatched_articles['year']-1

match = pd.merge(ml_unmatched_articles, imdb, 
                 left_on=['title', 'year+1'],
                 right_on=['originalTitle', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* fixed article + year mismatch")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

ml_unmatched_articles = ml_unmatched_articles[ml_unmatched_articles['movie_id'].isin(unmatched_movie_ids)]

# article notation change but match with title instead of originalTitle 

match = pd.merge(ml_unmatched_articles, imdb, 
                 left_on=['title', 'year+1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* fixed article + year mismatch")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

* fixed article + year mismatch
new matches: (17, 9)
all matches: (3398, 11)
left to match: (525, 5)
* fixed article + year mismatch
new matches: (1, 8)
all matches: (3399, 11)
left to match: (524, 5)


In [113]:
# let's try the +- year again 

ml_unmatched_articles = ml_unmatched_articles[ml_unmatched_articles['movie_id'].isin(unmatched_movie_ids)]

ml_unmatched_articles['year+1'] = ml_unmatched_articles['year']+1
ml_unmatched_articles['year-1'] = ml_unmatched_articles['year']-1

match = pd.merge(ml_unmatched_articles, imdb, 
                 left_on=['title', 'year-1'],
                 right_on=['originalTitle', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* fixed article + year mismatch")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

ml_unmatched_articles = ml_unmatched_articles[ml_unmatched_articles['movie_id'].isin(unmatched_movie_ids)]

# article notation change but match with title instead of originalTitle 

match = pd.merge(ml_unmatched_articles, imdb, 
                 left_on=['title', 'year-1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* fixed article + year mismatch")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

* fixed article + year mismatch
new matches: (20, 9)
all matches: (3419, 11)
left to match: (504, 5)
* fixed article + year mismatch
new matches: (1, 8)
all matches: (3420, 11)
left to match: (503, 5)


In [114]:
# no numbers + year

ml_unmatched_numbers = ml_unmatched.copy(deep='True')
ml_unmatched_numbers['year+1'] = ml_unmatched_numbers['year']+1
ml_unmatched_numbers['year+1'] = ml_unmatched_numbers['year']+1

imdb_numbers = imdb.copy(deep='True')

ml_unmatched_numbers['title'] = ml_unmatched_numbers['title'].apply(lambda x: remove_punctuation(x))
imdb_numbers['title'] = imdb_numbers['title'].apply(lambda x: remove_punctuation(x))

match = pd.merge(ml_unmatched_numbers, imdb_numbers, 
                 left_on=['title', 'year'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* no punctuation + year mismatch")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no punctuation + (year+1)

ml_unmatched_numbers = ml_unmatched.copy(deep='True')
ml_unmatched_numbers['year+1'] = ml_unmatched_numbers['year']+1

imdb_numbers = imdb.copy(deep='True')

ml_unmatched_numbers['title'] = ml_unmatched_numbers['title'].apply(lambda x: remove_punctuation(x))
imdb_numbers['title'] = imdb_numbers['title'].apply(lambda x: remove_punctuation(x))

match = pd.merge(ml_unmatched_numbers, imdb_numbers, 
                 left_on=['title', 'year+1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* no punctuation + year mismatch")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no punctuation + (year-1)

ml_unmatched_numbers = ml_unmatched.copy(deep='True')
ml_unmatched_numbers['year-1'] = ml_unmatched_numbers['year']-1

imdb_numbers = imdb.copy(deep='True')

ml_unmatched_numbers['title'] = ml_unmatched_numbers['title'].apply(lambda x: remove_punctuation(x))
imdb_numbers['title'] = imdb_numbers['title'].apply(lambda x: remove_punctuation(x))

match = pd.merge(ml_unmatched_numbers, imdb_numbers, 
                 left_on=['title', 'year-1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* no punctuation + year mismatch")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

* no punctuation + year mismatch
new matches: (28, 7)
all matches: (3448, 11)
left to match: (475, 5)
* no punctuation + year mismatch
new matches: (1, 8)
all matches: (3449, 11)
left to match: (474, 5)
* no punctuation + year mismatch
new matches: (1, 8)
all matches: (3450, 11)
left to match: (473, 5)


In [115]:
# no numbers + year

ml_unmatched_numbers = ml_unmatched.copy(deep='True')
ml_unmatched_numbers['year+1'] = ml_unmatched_numbers['year']+1
ml_unmatched_numbers['year+1'] = ml_unmatched_numbers['year']+1

imdb_numbers = imdb.copy(deep='True')

ml_unmatched_numbers['title'] = ml_unmatched_numbers['title'].apply(lambda x: remove_punctuation(x))
imdb_numbers['title'] = imdb_numbers['title'].apply(lambda x: remove_punctuation(x))

match = pd.merge(ml_unmatched_numbers, imdb_numbers, 
                 left_on=['title', 'year'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* no numbers + year mismatch")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no punctuation + (year+1)

ml_unmatched_numbers = ml_unmatched.copy(deep='True')
ml_unmatched_numbers['year+1'] = ml_unmatched_numbers['year']+1

imdb_numbers = imdb.copy(deep='True')

ml_unmatched_numbers['title'] = ml_unmatched_numbers['title'].apply(lambda x: remove_punctuation(x))
imdb_numbers['title'] = imdb_numbers['title'].apply(lambda x: remove_punctuation(x))

match = pd.merge(ml_unmatched_numbers, imdb_numbers, 
                 left_on=['title', 'year+1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* no numbers + year mismatch")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no punctuation + (year-1)

ml_unmatched_numbers = ml_unmatched.copy(deep='True')
ml_unmatched_numbers['year-1'] = ml_unmatched_numbers['year']-1

imdb_numbers = imdb.copy(deep='True')

ml_unmatched_numbers['title'] = ml_unmatched_numbers['title'].apply(lambda x: remove_punctuation(x))
imdb_numbers['title'] = imdb_numbers['title'].apply(lambda x: remove_punctuation(x))

match = pd.merge(ml_unmatched_numbers, imdb_numbers, 
                 left_on=['title', 'year-1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* no numbers + year mismatch")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

* no numbers + year mismatch
new matches: (0, 7)
all matches: (3450, 11)
left to match: (473, 5)
* no numbers + year mismatch
new matches: (0, 8)
all matches: (3450, 11)
left to match: (473, 5)
* no numbers + year mismatch
new matches: (0, 8)
all matches: (3450, 11)
left to match: (473, 5)


In [116]:
# secondary title

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year+1'] = ml_secondary_title['year']+1
ml_secondary_title['year+1'] = ml_secondary_title['year']+1


ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: extract_secondary(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]
print("* extract secondary title ")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no st + (year+1)

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year+1'] = ml_secondary_title['year']+1


ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: extract_secondary(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year+1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* extract secondary title ")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no punctuation + (year-1)

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year-1'] = ml_secondary_title['year']-1


ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: extract_secondary(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year-1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* extract secondary title ")

print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

* extract secondary title 
new matches: (20, 7)
all matches: (3470, 11)
left to match: (453, 5)
* extract secondary title 
new matches: (1, 8)
all matches: (3471, 11)
left to match: (452, 5)
* extract secondary title 
new matches: (1, 8)
all matches: (3472, 11)
left to match: (451, 5)


In [117]:
# secondary title

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year+1'] = ml_secondary_title['year']+1
ml_secondary_title['year+1'] = ml_secondary_title['year']+1


ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: remove_secondary(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* no secondary title")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no secondary + (year+1)

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year+1'] = ml_secondary_title['year']+1


ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: remove_secondary(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year+1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* no secondary title")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no punctuation + (year-1)

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year-1'] = ml_secondary_title['year']-1


ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: remove_secondary(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year-1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* no secondary title")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

* no secondary title
new matches: (109, 7)
all matches: (3581, 11)
left to match: (342, 5)
* no secondary title
new matches: (8, 8)
all matches: (3589, 11)
left to match: (334, 5)
* no secondary title
new matches: (8, 8)
all matches: (3597, 11)
left to match: (326, 5)


In [118]:
# secondary title + article reposition

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year+1'] = ml_secondary_title['year']+1
ml_secondary_title['year+1'] = ml_secondary_title['year']+1


ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: remove_secondary(x))
ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: reposition_movielens_article(x))


match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* secondary title + article reposition")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no punctuation + (year+1)

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year+1'] = ml_secondary_title['year']+1


ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: remove_secondary(x))
ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: reposition_movielens_article(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year+1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* secondary title + article reposition")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no punctuation + (year-1)

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year-1'] = ml_secondary_title['year']-1


ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: remove_secondary(x))
ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: reposition_movielens_article(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year-1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* secondary title + article reposition")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

* secondary title + article reposition
new matches: (60, 7)
all matches: (3657, 11)
left to match: (268, 5)
* secondary title + article reposition
new matches: (4, 8)
all matches: (3661, 11)
left to match: (264, 5)
* secondary title + article reposition
new matches: (3, 8)
all matches: (3664, 11)
left to match: (261, 5)


In [119]:
# replace & with and 

def replace_ampersand(title): 
    if ('&' in title): 
        title = title.replace('&', 'and')
# secondary title + article reposition

ml_secondary_title = ml_unmatched.copy(deep='True')

ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: replace_ampersand(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* remove ampersand")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no punctuation + (year+1)

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year+1'] = ml_secondary_title['year']+1

ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: replace_ampersand(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year+1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* remove ampersand")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no punctuation + (year-1)

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year-1'] = ml_secondary_title['year']-1

ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: replace_ampersand(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year-1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* remove ampersand")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

* remove ampersand
new matches: (0, 7)
all matches: (3664, 11)
left to match: (261, 5)
* remove ampersand
new matches: (0, 8)
all matches: (3664, 11)
left to match: (261, 5)
* remove ampersand
new matches: (0, 8)
all matches: (3664, 11)
left to match: (261, 5)


In [120]:
# remvove article from movielens 

ml_secondary_title = ml_unmatched.copy(deep='True')

ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: remove_movielens_articles(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* no article")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))


ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year+1'] = ml_secondary_title['year']+1

ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: remove_movielens_articles(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year+1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* no article")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))


ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year-1'] = ml_secondary_title['year']-1

ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: remove_movielens_articles(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year-1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("* no article")
print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

* no article
new matches: (17, 7)
all matches: (3681, 11)
left to match: (244, 5)
* no article
new matches: (1, 8)
all matches: (3682, 11)
left to match: (243, 5)
* no article
new matches: (2, 8)
all matches: (3684, 11)
left to match: (241, 5)


In [121]:
manually_fixing_df = pd.read_csv('https://raw.githubusercontent.com/jennyzhang0215/MovieLens-IMDB/master/movielens/statistics/manually_fixed_title_name', delimiter='|')

# remvove article from movielens 
# i don't think this works
def replace_manually(title): 
    fixed_title = title
    if (len(manually_fixing_df[manually_fixing_df['wrong_title_name'] == title] == 1)): 
        fixed_title = manually_fixing_df[manually_fixing_df['wrong_title_name'] == title]['correct_title_name'].values[0]
    return fixed_title

# secondary title + article reposition

ml_secondary_title = ml_unmatched.copy(deep='True')

ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: replace_manually(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no punctuation + (year+1)

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year+1'] = ml_secondary_title['year']+1

ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: remove_movielens_articles(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year+1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

# no punctuation + (year-1)

ml_secondary_title = ml_unmatched.copy(deep='True')
ml_secondary_title['year-1'] = ml_secondary_title['year']-1

ml_secondary_title['title'] = ml_secondary_title['title'].apply(lambda x: remove_movielens_articles(x))

match = pd.merge(ml_secondary_title, imdb, 
                 left_on=['title', 'year-1'],
                 right_on=['title', 'year'])

matched_df_full = matched_df_full.append(match, ignore_index=True)

matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]

print("new matches: " + str(match.shape))
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))

new matches: (51, 7)
all matches: (3735, 11)
left to match: (190, 5)
new matches: (0, 8)
all matches: (3735, 11)
left to match: (190, 5)
new matches: (0, 8)
all matches: (3735, 11)
left to match: (190, 5)


In [122]:
manual_matching_dict = {'dream man': 'tt0101770', 
               'new york cop': 'tt0368893',
               'true crime': 'tt0139668',
               'costa brava': 'tt0109489',
               'victor/victoria': 'tt0265987',
               'drunks': 'tt0112907',
               'farmer & chase': 'tt0113031',
               'kids of survival': 'tt0107314',
               'blood & wine': 'tt0859643',
               'hearts and minds': 'tt0071604',
               'alien escape': 'tt0112318',
               'other voices, other rooms': 'tt0119845',
               'time tracers': 'tt0128755',
               'follow the bitch': 'tt0119139',
               '101 dalmatians': 'tt011543',
               'steamboat willie': 'tt0019422',
               'henry: portrait of a serial killer': 'tt0099763',
               'attack of the killer tomatoes!': 'tt0080391',
               'little nemo: adventures in slumberland': 'tt0104740',
               'ten benny': 'tt0114008',
               'daddy long legs': 'tt0021775',
               'train ride to hollywood': 'tt0078412',
               'santitos': 'tt0126651',
               'it happened here': 'tt0055024',
               'last resort': 'tt0091387',
               'solar crisis': 'tt0100649',
               'kronos': 'tt0050610',
               'misérables, les': 'tt0077936', 
               'shadows (cienie)': 'tt0245718',
               'castle freak': 'tt10701458',
               'dumb & dumber': 'tt0109686',
               'farinelli: il castrato': 'tt0109771',
               'interview with the vampire': 'tt1860252',
               'enfer l\'': 'tt13124824',
               'robert a. heinlein\'s the puppet masters': 'tt0111003',
               'harlem': 'tt0034950',
               'wedding gift, the': 'tt0847585',
               'ciao, professore! (io speriamo che me la cavo )': 'tt0107225',
               'dear diary (caro diario)': 'tt0109382',
               'superweib, das': 'tt0117788',
               'promise, the (versprechen, das)': 'tt0111613',
               'under the domin tree (etz hadomim tafus)': 'tt0109751',
               'two friends': '1986',
               'rendezvous in paris (rendez-vous de paris, les)': 'tt0176090',
               'crude oasis, the': 'tt0112746',
               'godzilla 2000 (gojira ni-sen mireniamu)': 'tt0120685',
               'broken hearts club, the': 'tt1194103',
               'crime and punishment in suburbia': 'tt0096056',
               'mad max 2 (a.k.a. the road warrior)': 'tt0079501',
               'toxic avenger, part ii, the': 'tt0090190',
               'spring fever usa (a.k.a. lauderdale)': 'tt0097717',
               'i am cuba (soy cuba/ya kuba)': 'tt0058604',
               'wisdom of crocodiles, the (a.k.a. immortality)': 'tt0120894',
               'mr. death: the rise and fall of fred a. leuchter, jr.': 'tt0192335',
               'pink floyd - the wall': 'tt0084503', 
               'nosferatu a venezia': 'tt0091651',
               'good, the bad and the ugly, the': 'tt5083572',
               'two women (la ciociara)': 'tt0054749',
               'robert a. heinlein\'s the puppet masters': 'tt0111003',
                'the players club':'tt0119905',
                'big bang theory, the': 'tt1147717',
'jungle2jungle (a.k.a. jungle 2 jungle)' : 'tt0119432',
'boys, les' : 'tt0118764',
'prophecy ii, the' : 'tt0114194',
'machine, the':  'tt0933079',
'friday the 13th part 3: 3d' : 'tt0080761',
'karate kid, part ii, the' : 'tt0426060',
'empty mirror, the':  'tt0116192',
'citizen\'s band (a.k.a. handle with care)':  'tt0359987',
'hard 8 (a.k.a. sydney, a.k.a. hard eight)' : 'tt0119256' ,
'poison ivy: new seduction' : 'tt0105156' ,
'hard-boiled (lashou shentan)' : 'tt0104684',
'trial, the (le procés)':  'tt0057427',
'horror hotel (a.k.a. the city of the dead)' : 'tt0053719',
'two or three things i know about her' : 'tt0060304',
'vacation' : 'tt0015452',
'slaughterhouse 2':  'tt0093990'	,
'meatballs iii':  'tt0079540',
'children of the corn iii':  'tt0087050',
'seven beauties (pasqualino settebellezze)': 'tt0075040',
'lodger, the':  'tt0037024',
'vie est belle, la (life is rosey)' : 'tt0161066',
'communion (a.k.a. alice, sweet alice/holy terror)' : 'tt0188223',}
               

for index, row in ml_unmatched.iterrows():
    title = row['title']
    if(manual_matching_dict.get(title) != None): 
        row['tconst'] = manual_matching_dict.get(title)
        matched_df_full = matched_df_full.append(row)
        
matched_movie_ids = list(matched_df_full['movie_id'])
unmatched_movie_ids = set(unmatched_movie_ids) - set(matched_movie_ids)

ml_unmatched = ml_unmatched[ml_unmatched['movie_id'].isin(list(unmatched_movie_ids))]   
        
print("all matches: " + str(matched_df_full.shape))
print("left to match: " + str(ml_unmatched.shape))


all matches: (3808, 11)
left to match: (117, 5)


In [123]:
match_without_year = pd.merge(ml_unmatched, imdb, 
                 on=['title'])

In [124]:
match_without_year

,movie_id,title,year_x,year+1,year-1,tconst,originalTitle,year_y
0,28,persuasion,1995,1996,1994,tt0844330,persuasion,2007
1,28,persuasion,1995,1996,1994,tt1217062,persuasion,2008
2,51,guardian angel,1994,1995,1993,tt0173642,andjeo cuvar,1987
3,51,guardian angel,1994,1995,1993,tt2992538,guardian angel,2014
4,51,guardian angel,1994,1995,1993,tt5228142,guardian angel,2016
5,51,guardian angel,1994,1995,1993,tt8145386,guardian angel,2019
6,2311,2010,1984,1985,1983,tt0883356,2010,2006
7,3660,puppet master,1989,1990,1988,tt1815887,puppet master,2010
8,3660,puppet master,1989,1990,1988,tt2331990,puppet master,2012
9,3660,puppet master,1989,1990,1988,tt8164188,puppet master,2018


In [125]:
match_without_year = pd.merge(ml_unmatched, imdb, 
                 left_on=['title'], right_on=['originalTitle'])

In [126]:
match_without_year

,movie_id,title_x,year_x,year+1,year-1,tconst,title_y,originalTitle,year_y
0,28,persuasion,1995,1996,1994,tt0844330,persuasion,persuasion,2007
1,28,persuasion,1995,1996,1994,tt1217062,persuasion,persuasion,2008
2,51,guardian angel,1994,1995,1993,tt2992538,guardian angel,guardian angel,2014
3,51,guardian angel,1994,1995,1993,tt5228142,guardian angel,guardian angel,2016
4,51,guardian angel,1994,1995,1993,tt8145386,guardian angel,guardian angel,2019
5,2311,2010,1984,1985,1983,tt0883356,2010,2010,2006
6,3660,puppet master,1989,1990,1988,tt1815887,puppet master,puppet master,2010
7,3660,puppet master,1989,1990,1988,tt2331990,puppet master,puppet master,2012
8,3660,puppet master,1989,1990,1988,tt8164188,puppet master,puppet master,2018


In [127]:
matched_df_full = matched_df_full.drop(columns=['originalTitle', 'title_x', 'title_y', 'year_x','year+1','year-1', 'year_y'])

In [128]:
matched_df_full.head()

,movie_id,title,year,tconst
0,1,toy story,1995.00000,tt0114709
1,2,jumanji,1995.00000,tt0113497
2,3,grumpier old men,1995.00000,tt0113228
3,4,waiting to exhale,1995.00000,tt0114885
4,5,father of the bride part ii,1995.00000,tt0113041


In [130]:
matched_df_full.to_csv('matched_movies_ML_IMDB.csv', index=False)